In [7]:
import numpy as np
import pandas as pd

from scipy import stats
import itertools
from sklearn import linear_model
from numpy import ones,vstack
from numpy.linalg import lstsq

pd.options.display.max_rows=400

In [8]:
df=pd.read_csv('./data/ames_housing_price_data_v2.csv', index_col=0)

In [ ]:
overallcondition - maybe
anything with 'cond' - maybe
maybe_Exterior1st	drop_Exterior2nd	aggregate and dummify
EqBath_0.3,0.5.... -> drop
Mo/Yr sold -> make datetime object
LF_Normal -> drop
BsmtCond_ord_lin & others -> drop for now

SalePrice -> add.separate log column?


In [9]:
df.head().T

PID,909176150,905476230,911128020,535377150,534177230
GrLivArea,856,1049,1001,1039,1665
SalePrice,126000,139500,124900,114000,227000
drop_MSSubClass,30,120,30,70,60
maybe_MSZoning,RL,RL,C (all),RL,RL
LotFrontage,0.0,42.0,60.0,80.0,70.0
LotArea,7890,4235,6060,8146,8400
Street_paved,Pave,Pave,Pave,Pave,Pave
Alley,No alley access,No alley access,No alley access,No alley access,No alley access
maybe_LotShape,Reg,Reg,Reg,Reg,Reg
LandContour,Level,Level,Level,Level,Level
